In [1]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import math
import time
from utils.utils import * 
from utils.rf_gaussians import gaussian_2d_curve

In [2]:
columns = ["x0", "y0", "sigma", "slope", "intercept"]

In [3]:
betas_file = os.path.join(proj_dir, 'betas', 'subj01', 'subj01_37_betas_train_nativesurface.npy')
betas_01 = np.load(betas_file, allow_pickle=False).astype(np.float32).T
n_betas, n_voxels = betas_01.shape

In [7]:
# Luis ver1 
params = {'random': True, 
                    'initial': (np.array([-0.4, -0.4, 0.01, 0.1, - 2]), np.array([0.4, 0.4, 2, 10, 2])), 
                    'bounds': (np.array([-1.05, -1.05, 0.01, 1e-08, - np.inf]),np.array([1.05, 1.05, np.inf, np.inf, np.inf])),
                    'loss': 'linear',
                    'method': 'trf'
                   }

initial = params['initial']
bounds = params['bounds']

In [9]:
for roi in rois.keys():
    start = time.time()
    mds_file = os.path.join(proj_dir, 'MDS_rotated', 'subj01', f'subj01_37_{roi}_mds_betas_train_rotated.npy')
    mds = np.load(mds_file, allow_pickle=True).astype(np.float32).T
    print('Rotated MDS loaded')
    fit_file = os.path.join(proj_dir, 'fits_rotated', 'subj01')
    if not os.path.exists(fit_file):
        os.makedirs(fit_file)
    fit_file = os.path.join(fit_file, f'fits_subj01_37_trains_version1_{roi}.npy')
    if os.path.exists(fit_file):
        print(f'\t\t\ŧskipping {roi}, already exists')
        continue
    fits_roi = pd.DataFrame(columns=columns)

    print(f'Starting fitting for ROI {roi}, on version 1')
    for voxel in range(n_voxels):
        if not voxel % 1000:
            print(
                f'\t\t\t\t\t[{100*voxel/n_voxels:.2f}%] elapsed time since {roi} start: ',
                            f'{time.strftime("%H:%M:%S", time.gmtime(time.time() - start))}'
                        )
        voxel_activity = betas_01[:, voxel]
        if params['random']:
            attempt = 1
            solved = False
            while not solved and attempt <= 10:
                try:
                    initial_guess = (initial[1] - initial[0]) * np.random.random(initial[0].shape) + initial[0]
                    voxel_fit = curve_fit(
                        gaussian_2d_curve,
                        mds,
                        voxel_activity,
                        p0  = initial_guess,
                        bounds=params['bounds'],
                        method = 'trf',
                        ftol = 1e-06
                    )[0]
                    solved = True
                except RunetimeError:
                    print(f'VOXEL {voxel}: optimal params not found after {attempt} attempts')
                    attempt +1
    def gaus_roi(fits):
        return gaussian_2d_curve(md, *fits)

    pred_activity = fits_roi.apply(gaus_roi, axis = 1)
    pred_activity = np.array([np.array(x) for x in pred_activity]).T
    roi_res = np.sum((pred_activity - betas_01)**2, axis=0)
    roi_tot = sum((betas- np.tile(betas_01.mean(axis=0), (n_betas, 1)))**2).T

    ## Maybe include split here

    fits_roi["var_explained"] = 1 - roi_res / roi_tot
    fits_roi["mds_ecc"] = (fits_roi.x0 ** 2 + fits_roi.y0[1] ** 2) ** (1/2)
    fits_roi["mds_ang"] = np.arctan2(fits_roi.x0/bounds[1][0], fits_roi.y0/bounds[1][1])

    np.save(fit_file, fits_roi)
    print(f'\t\t\tTime elapsed during {roi}: ',
    f'{time.strftime("%H:%M:%S", time.gmtime(time.time() - start))}'
                )
del betas_01
                                     
    

Rotated MDS loaded
Starting fitting for ROI V1, on version 1
					[0.00%] elapsed time since V1 start:  00:00:00
					[2.96%] elapsed time since V1 start:  00:00:47


NameError: name 'RunetimeError' is not defined